In [16]:
from ortools.linear_solver import pywraplp

# -----------------------
# Sample Data (customize)
# -----------------------

# Set of demand points
I = [0, 1, 2, 3]  # Demand point indices

# Set of potential facility locations
J = [0, 1, 2, 3]  # Facility indices

# J_i: Facilities in range of each demand point i
J_i = {
    0: [0, 1],
    1: [0, 1, 2],
    2: [1, 2],
    3: [2, 3]
}

# Weight (or demand value) for each demand point
w = {
    0: 10,
    1: 20,
    2: 30,
    3: 40
}

# Facility capacity
U = 50

# Max number of facilities to open
p = 2

# -----------------------
# Create Solver
# -----------------------

solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    raise Exception("Solver not created.")

# -----------------------
# Decision Variables
# -----------------------

# x[i][j] = 1 if demand i is assigned to facility j
x = {}
for i in I:
    for j in J_i[i]:
        x[i, j] = solver.BoolVar(f'x_{i}_{j}')

# y[j] = 1 if facility j is opened
y = {}
for j in J:
    y[j] = solver.BoolVar(f'y_{j}')

# -----------------------
# Objective Function
# -----------------------

# Maximize total weight of assigned demand
solver.Maximize(solver.Sum(w[i] * x[i, j] for i in I for j in J_i[i]))

# -----------------------
# Constraints
# -----------------------

# (11) Each demand point assigned to at most one facility in range
for i in I:
    solver.Add(solver.Sum(x[i, j] for j in J_i[i]) <= 1)

# (12) At most p facilities can be opened
solver.Add(solver.Sum(y[j] for j in J) <= p)

# (13) Total demand assigned to facility j cannot exceed capacity U
for j in J:
    # Only consider x[i, j] where j is in J_i[i]
    assigned_demand = [w[i] * x[i, j] for i in I if j in J_i[i]]
    if assigned_demand:
        solver.Add(solver.Sum(assigned_demand) <= U)

# (14) Demand point can only be assigned to open facilities
for i in I:
    for j in J_i[i]:
        solver.Add(x[i, j] <= y[j])

# (15) Binary variables: already ensured via BoolVar

# -----------------------
# Solve the Problem
# -----------------------

status = solver.Solve()

# -----------------------
# Output Results
# -----------------------

if status == pywraplp.Solver.OPTIMAL:
    print("Solution found:")
    for j in J:
        if y[j].solution_value() > 0.5:
            print(f"Facility {j} is OPEN")
    for i in I:
        for j in J_i[i]:
            if x[i, j].solution_value() > 0.5:
                print(f"Demand point {i} assigned to facility {j}")
else:
    print("No optimal solution found.")


Solution found:
Facility 1 is OPEN
Facility 2 is OPEN
Demand point 1 assigned to facility 1
Demand point 2 assigned to facility 1
Demand point 3 assigned to facility 2


In [17]:
def stage_2_knapsack_assignment(open_facilities, demand_assignments, w, b, B, K):
    """
    Assign drones to facilities and demand locations using repeated knapsack problems.

    Args:
        open_facilities (list): List of open facilities (j).
        demand_assignments (dict): Keys are facility indices j, values are demand indices i assigned to j.
        w (dict): Demand weight {i: value}.
        b (dict): Battery usage/distance { (i, j): distance }.
        B (int): Drone battery capacity.
        K (int): Total number of drones.

    Returns:
        dict: Mapping from drone index to assigned (facility, demand_list)
    """
    from ortools.linear_solver import pywraplp

    drones_used = 0
    drone_assignments = {}
    Ij = {j: list(demand_assignments[j]) for j in open_facilities}  # Copy of demand sets

    while drones_used < K and any(Ij[j] for j in open_facilities):
        best_j = None
        best_x = {}
        best_value = -1

        for j in open_facilities:
            items = Ij[j]
            if not items:
                continue

            solver = pywraplp.Solver.CreateSolver('SCIP')
            if not solver:
                raise Exception("Solver not created.")

            x = {}
            for i in items:
                x[i] = solver.BoolVar(f'x_{i}')

            # Objective: Maximize total demand weight
            solver.Maximize(solver.Sum(w[i] * x[i] for i in items))

            # Constraint: total distance ≤ battery limit
            solver.Add(solver.Sum(b[i, j] * x[i] for i in items) <= B)

            status = solver.Solve()

            if status == pywraplp.Solver.OPTIMAL:
                total_value = sum(w[i] for i in items if x[i].solution_value() > 0.5)
                if total_value > best_value:
                    best_value = total_value
                    best_j = j
                    best_x = {i: x[i].solution_value() > 0.5 for i in items}

        if best_j is None:
            break  # No feasible assignment left

        assigned_i = [i for i, assigned in best_x.items() if assigned]
        if not assigned_i:
            break  # No assignment possible with remaining demands

        drone_assignments[drones_used] = {
            "facility": best_j,
            "demands": assigned_i
        }

        # Remove assigned demands from Ij
        for i in assigned_i:
            Ij[best_j].remove(i)

        drones_used += 1

    return drone_assignments


In [18]:
import random
import numpy as np
from itertools import combinations
from collections import defaultdict

# Continuing from previous context: using facilities_opened, demand_assignments, and drone_assignments

def solve_allocation_fixed_facilities(I, J_fixed, demand_weights, facility_capacities, facility_demand_range, drone_range_matrix):
    """
    Re-solve allocation problem where facility locations are fixed (y_j = 1 if j in J_fixed).
    """
    new_demand_assignments = {}
    facility_loads = {j: 0 for j in J_fixed}

    for i in I:
        candidates = [j for j in J_fixed if drone_range_matrix[i][j] == 1]
        candidates = sorted(candidates, key=lambda j: -demand_weights[i])
        for j in candidates:
            if facility_loads[j] + demand_weights[i] <= facility_capacities[j]:
                new_demand_assignments[i] = j
                facility_loads[j] += demand_weights[i]
                break

    return new_demand_assignments

def knapsack_assignment(Ij, drone_range_matrix, demand_weights, battery_limit):
    """
    Solve 0-1 knapsack for a single facility.
    """
    best_value = 0
    best_subset = []

    n = len(Ij)
    for r in range(1, n + 1):
        for subset in combinations(Ij, r):
            total_weight = sum(demand_weights[i] for i in subset)
            if all(drone_range_matrix[i] == 1 for i in subset) and total_weight > best_value and total_weight <= battery_limit:
                best_value = total_weight
                best_subset = list(subset)

    return best_subset, best_value

def stage3_r_exchange(
    I, J, demand_weights, facility_capacities, drone_range_matrix,
    facilities_opened, demand_assignments, K, battery_limit,
    num_iterations=10, r=1
):
    J0 = facilities_opened.copy()
    best_total_demand = 0
    best_assignment = {}
    best_facilities = J0.copy()

    for _ in range(num_iterations):
        # Calculate total demand per facility
        facility_totals = {j: 0 for j in J0}
        for i, j in demand_assignments.items():
            if j in J0:
                facility_totals[j] += demand_weights[i]

        # Identify r lowest demand facilities
        to_remove = sorted(J0, key=lambda j: facility_totals[j])[:r]
        J_candidate_pool = list(set(J) - set(J0))
        if len(J_candidate_pool) < r:
            break  # Not enough to replace

        to_add = random.sample(J_candidate_pool, r)
        J_new = list(set(J0) - set(to_remove)) + to_add

        # Re-define range Ji for each demand i
        demand_to_J = {
            i: [j for j in J_new if drone_range_matrix[i][j] == 1] for i in I
        }

        # Re-solve allocation problem with fixed facilities
        new_demand_assignments = solve_allocation_fixed_facilities(
            I, J_new, demand_weights, facility_capacities, demand_to_J, drone_range_matrix
        )

        # Stage 2 again: assign drones using knapsack
        Ij = defaultdict(list)
        for i, j in new_demand_assignments.items():
            Ij[j].append(i)

        assigned = set()
        drone_assignment = defaultdict(list)
        for k in range(K):
            best_val = -1
            best_facility = None
            best_subset = []

            for j in J_new:
                unassigned = [i for i in Ij[j] if i not in assigned]
                subset, val = knapsack_assignment(unassigned, drone_range_matrix[j], demand_weights, battery_limit)
                if val > best_val:
                    best_val = val
                    best_facility = j
                    best_subset = subset

            if best_facility is not None:
                drone_assignment[best_facility].append(best_subset)
                assigned.update(best_subset)

        total_demand_served = sum(demand_weights[i] for i in assigned)

        if total_demand_served > best_total_demand:
            best_total_demand = total_demand_served
            best_assignment = drone_assignment
            best_facilities = J_new
            J0 = J_new

    return best_total_demand, best_assignment, best_facilities



In [19]:
def solve_allocation_fixed_facilities(I, J_fixed, w, U, drone_range_matrix):
    new_assignments = {}
    loads = {j: 0 for j in J_fixed}
    for i in I:
        candidates = [j for j in J_fixed if drone_range_matrix[i][j] == 1]
        for j in candidates:
            if loads[j] + w[i] <= U:
                new_assignments[i] = j
                loads[j] += w[i]
                break
    return new_assignments

def knapsack_assignment(items, w, B):
    best_value = 0
    best_subset = []
    n = len(items)
    for r in range(1, n + 1):
        for subset in combinations(items, r):
            total_weight = sum(w[i] for i in subset)
            if total_weight > best_value and total_weight <= B:
                best_value = total_weight
                best_subset = list(subset)
    return best_subset, best_value

def stage3_r_exchange(
    I, J, w, U, drone_range_matrix,
    facilities_opened, demand_assignments, K, B,
    num_iterations=5, r=1
):
    J0 = facilities_opened.copy()
    best_total_demand = sum(w[i] for i in demand_assignments)
    best_assignment = demand_assignments.copy()
    best_facilities = J0.copy()
    best_drone_assignment = {}

    for _ in range(num_iterations):
        facility_totals = {j: 0 for j in J0}
        for i, j in best_assignment.items():
            if j in J0:
                facility_totals[j] += w[i]

        to_remove = sorted(J0, key=lambda j: facility_totals[j])[:r]
        candidate_pool = list(set(J) - set(J0))
        if len(candidate_pool) < r:
            break
        to_add = random.sample(candidate_pool, r)
        J_new = list(set(J0) - set(to_remove)) + to_add

        new_demand_assignments = solve_allocation_fixed_facilities(I, J_new, w, U, drone_range_matrix)

        Ij = defaultdict(list)
        for i, j in new_demand_assignments.items():
            Ij[j].append(i)

        assigned = set()
        drone_assignment = defaultdict(list)

        for _ in range(K):
            best_val = -1
            best_fac = None
            best_subset = []

            for j in J_new:
                unassigned = [i for i in Ij[j] if i not in assigned]
                subset, val = knapsack_assignment(unassigned, w, B)
                if val > best_val:
                    best_val = val
                    best_fac = j
                    best_subset = subset

            if best_subset:
                drone_assignment[best_fac].append(best_subset)
                assigned.update(best_subset)

        total_demand_served = sum(w[i] for i in assigned)

        if total_demand_served > best_total_demand:
            best_total_demand = total_demand_served
            best_assignment = new_demand_assignments
            best_facilities = J_new
            best_drone_assignment = drone_assignment
            J0 = J_new

    return best_total_demand, best_assignment, best_facilities, best_drone_assignment

# Sample data setup (using variables from earlier stages)
I = [0, 1, 2, 3]  # Demand points
J = [0, 1, 2, 3]  # Facilities
w = {0: 10, 1: 20, 2: 30, 3: 40}  # Demand weights
U = 50  # Facility capacity
drone_range_matrix = {
    0: {0: 1, 1: 1, 2: 0, 3: 0},
    1: {0: 1, 1: 1, 2: 1, 3: 0},
    2: {0: 0, 1: 1, 2: 1, 3: 0},
    3: {0: 0, 1: 0, 2: 1, 3: 1}
}
facilities_opened = [0, 1]  # Initial opened facilities
demand_assignments = {0: 0, 1: 0, 2: 1, 3: 1}  # Initial assignments
K = 3  # Number of drones
B = 50  # Battery capacity

# Run the optimization
best_total_demand, best_assignments, best_facilities, best_drone_assignments = stage3_r_exchange(
    I, J, w, U, drone_range_matrix,
    facilities_opened, demand_assignments, K, B,
    num_iterations=5, r=1
)

# Print results
print("\nStage 3 - r-exchange Heuristic Results:")
print("-------------------------------------")
print(f"Best total demand served: {best_total_demand}")
print(f"Best facilities opened: {best_facilities}")

print("\nDemand assignments:")
for i, j in best_assignments.items():
    print(f"Demand point {i} (weight={w[i]}) → Facility {j}")

print("\nDrone assignments:")
for facility, assignments in best_drone_assignments.items():
    for idx, drone_tour in enumerate(assignments):
        total_weight = sum(w[i] for i in drone_tour)
        print(f"Drone {idx+1} at Facility {facility} serves: {drone_tour} (Total weight: {total_weight}/{B})")


Stage 3 - r-exchange Heuristic Results:
-------------------------------------
Best total demand served: 100
Best facilities opened: [0, 1]

Demand assignments:
Demand point 0 (weight=10) → Facility 0
Demand point 1 (weight=20) → Facility 0
Demand point 2 (weight=30) → Facility 1
Demand point 3 (weight=40) → Facility 1

Drone assignments:
